# Première itération avec poisson

année, loca, nb restau

In [2]:
# ============================================================
# Trend par localité (Poisson + CAGR) à partir d'un CSV
# ============================================================
# Dépendances : pandas, numpy, statsmodels
# pip install pandas numpy statsmodels

import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
from statsmodels.tools.sm_exceptions import PerfectSeparationError

# ---------- Paramètres ----------
CSV_PATH = "restaurant_count_by_locality.csv"   # <- chemin vers ton CSV
YEAR_MIN, YEAR_MAX = 2011, 2025                 # plage d'années à analyser (inclusives)
EXCLUDE_YEAR = []                               # ex.: [2020] pour exclure 2020

# ---------- Chargement ----------
df = pd.read_csv(CSV_PATH)

# Normalisation de colonnes (au cas où)
df.columns = [c.strip().lower() for c in df.columns]
expected_cols = {"locality", "year", "num_restaurants"}
missing = expected_cols - set(df.columns)
if missing:
    raise ValueError(
        f"Colonnes manquantes dans le CSV: {missing}. "
        f"Colonnes trouvées: {list(df.columns)}"
    )

# Filtrage éventuel des années
mask = (df["year"].between(YEAR_MIN, YEAR_MAX)) & (~df["year"].isin(EXCLUDE_YEAR))
df = df.loc[mask].copy()

# Sécurité : comptes >= 0 (Poisson)
df = df[df["num_restaurants"] >= 0].copy()

# ---------- Exclure localités avec < 3 années distinctes ----------
year_counts = df.groupby("locality")["year"].nunique()
excluded_low_years = year_counts[year_counts < 3].index.tolist()

if excluded_low_years:
    print(f"⛔ Exclues (moins de 3 années): {len(excluded_low_years)} localités")

df_fit = df[~df["locality"].isin(excluded_low_years)].copy()

# Stat des séries constantes (toutes années = même compte)
n_const_total = (
    df_fit.groupby("locality")["num_restaurants"].nunique().eq(1).sum()
)
print(f"ℹ️ Localités à série constante (aucune tendance attendue): {n_const_total}")

# Liste pour tracer les échecs de fit (par ex. singularités)
failed_fits = []

def poisson_trend_for_group(g: pd.DataFrame) -> pd.Series:
    """
    Calcule des indicateurs de tendance pour une localité donnée :
    - Annual Growth % (Poisson GLM): exp(beta_year) - 1
    - CAGR % entre la 1ère et la dernière année observées
    - Deviance explained (pseudo-R²)
    - p-value du coef de l'année
    - trend_score = (Annual Growth %) * max(0, deviance_explained)

    Hypothèse: g contient au moins 3 années distinctes (filtré en amont).
    """
    # Agrège si jamais il y avait des doublons (locality, year)
    g = (
        g.sort_values("year")
         .groupby(["locality", "year"], as_index=False)["num_restaurants"]
         .sum()
         .sort_values("year")
    )

    loc = g["locality"].iloc[0]

    # Bornes / stats de base
    first_year = int(g["year"].iloc[0])
    last_year  = int(g["year"].iloc[-1])
    n_years = last_year - first_year
    first_val = float(g["num_restaurants"].iloc[0])
    last_val  = float(g["num_restaurants"].iloc[-1])

    # ---------- 1) Cas "constant" : tous les comptes identiques ----------
    if g["num_restaurants"].nunique() == 1:
        const_val = float(g["num_restaurants"].iloc[0])
        # CAGR : 0% si durée>0 et valeur constante >0 ; sinon NaN (ex.: tout 0)
        if n_years > 0 and const_val > 0:
            cagr_pct = 0.0
        else:
            cagr_pct = np.nan

        return pd.Series({
            "locality": loc,
            "years_covered": f"{first_year}-{last_year}",
            "n_obs": len(g),
            "annual_growth_pct_poisson": 0.0,    # pente nulle (exp(0)-1)
            "cagr_pct": cagr_pct,                # 0 si constant>0, sinon NaN
            "p_value_year": 1.0,                 # pas d'effet détectable
            "deviance_explained": 0.0,           # rien à expliquer
            "trend_score": 0.0                   # 0 * 0 = 0
        })

    # ---------- 2) Cas "non-constant" : on fit le GLM ----------
    # CAGR observé entre première et dernière année
    if n_years > 0 and first_val > 0:
        cagr_pct = ((last_val / first_val) ** (1 / n_years) - 1.0) * 100.0
    else:
        cagr_pct = np.nan

    # Par défaut (au cas où le GLM échoue)
    annual_growth_pct = np.nan
    pval_year = np.nan
    dev_explained = 0.0   # par défaut 0.0 (plus stable)
    trend_score = np.nan

    # Design (centrage de l'année pour robustesse num.)
    year_centered = g["year"] - g["year"].mean()
    X = pd.DataFrame({"const": 1.0, "year": year_centered.astype(float)})
    y = g["num_restaurants"].astype(float)

    model = sm.GLM(y, X, family=sm.families.Poisson())

    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            use_hc3 = (len(g) - X.shape[1]) > 0  # df_resid>0
            res = model.fit(cov_type="HC3") if use_hc3 else model.fit()

        beta_year = res.params.get("year", np.nan)
        pval_year = res.pvalues.get("year", np.nan) if hasattr(res, "pvalues") else np.nan

        if np.isfinite(beta_year):
            annual_growth_pct = (np.exp(beta_year) - 1.0) * 100.0

        # Pseudo-R² via deviance expliquée
        dev = getattr(res, "deviance", np.nan)
        null_dev = getattr(res, "null_deviance", np.nan)
        if np.isfinite(dev) and np.isfinite(null_dev) and null_dev > 0:
            dev_explained = max(0.0, 1.0 - (dev / null_dev))
        else:
            dev_explained = 0.0

        if np.isfinite(annual_growth_pct):
            trend_score = annual_growth_pct * dev_explained

    except (PerfectSeparationError, np.linalg.LinAlgError, ValueError) as e:
        failed_fits.append((loc, str(e)))

    return pd.Series({
        "locality": loc,
        "years_covered": f"{first_year}-{last_year}",
        "n_obs": len(g),
        "annual_growth_pct_poisson": annual_growth_pct,
        "cagr_pct": cagr_pct,
        "p_value_year": pval_year,
        "deviance_explained": dev_explained,
        "trend_score": trend_score
    })

# ---------- Calcul par localité ----------
if df_fit.empty:
    print("⚠️ Aucun groupe à traiter après filtrage (années / exclusions).")
    trend_table_sorted = pd.DataFrame(
        columns=[
            "locality","years_covered","n_obs",
            "annual_growth_pct_poisson","cagr_pct",
            "p_value_year","deviance_explained","trend_score"
        ]
    )
else:
    trend_table = (
        df_fit.groupby("locality", group_keys=False)
              .apply(poisson_trend_for_group)
              .reset_index(drop=True)
    )

    # Tri par score décroissant (plus haut = plus dynamique)
    trend_table_sorted = trend_table.sort_values("trend_score", ascending=False)

# ---------- Affichage et sauvegarde ----------
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")
print("\nAperçu des tendances (triées par score décroissant) :")
print(trend_table_sorted.head(20).to_string(index=False))

OUT_CSV = "locality_trends_poisson.csv"
trend_table_sorted.to_csv(OUT_CSV, index=False)
print(f"\n📄 Résultats exportés → {OUT_CSV}")

# --------- (Optionnel) filtrage des tendances significatives ----------
if not trend_table_sorted.empty:
    signif_10 = trend_table_sorted[
        (trend_table_sorted["p_value_year"].notna()) &
        (trend_table_sorted["p_value_year"] < 0.10)
    ]
    print(f"\nLocalités avec tendance significative (p<0.10): {len(signif_10)}")
    print(signif_10[[
        "locality","annual_growth_pct_poisson","cagr_pct","p_value_year","trend_score"
    ]].head(20).to_string(index=False))

# ---------- Journal des exclusions / échecs ----------
if excluded_low_years:
    print("\n⛔ Localités exclues (moins de 3 années distinctes):")
    print(", ".join(sorted(set(excluded_low_years))))

if failed_fits:
    print("\n⚠️  Localités non traitées (échec du fit Poisson) + message d'erreur:")
    for loc, msg in failed_fits[:50]:
        print(f"- {loc}: {msg}")
    if len(failed_fits) > 50:
        print(f"... (+{len(failed_fits)-50} supplémentaires)")


⛔ Exclues (moins de 3 années): 72 localités
ℹ️ Localités à série constante (aucune tendance attendue): 451

Aperçu des tendances (triées par score décroissant) :
          locality years_covered  n_obs  annual_growth_pct_poisson  cagr_pct  p_value_year  deviance_explained  trend_score
         Rougemont     2023-2025      3                   130.2776  100.0000        0.0000              0.8543     111.2918
Torricella-Taverne     2023-2025      3                    89.6805   73.2051        0.0000              0.8350      74.8855
         Fontenais     2023-2025      3                    46.8375   41.4214        0.0000              0.8055      37.7294
            Soazza     2023-2025      3                    46.8375   41.4214        0.0000              0.8055      37.7294
           Bottens     2022-2025      4                    52.1380   44.2250        0.0002              0.7118      37.1124
          Kappelen     2023-2025      3                    55.6466   73.2051        0.0034    

/tmp/ipykernel_53190/1070982890.py:174: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(poisson_trend_for_group)


In [3]:
# ============================================================
# Trend par localité (Poisson + CAGR) — sortie minimaliste
# -> ne conserve que locality et trend_score
# ============================================================
# Dépendances : pandas, numpy, statsmodels
# pip install pandas numpy statsmodels

import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
from statsmodels.tools.sm_exceptions import PerfectSeparationError

# ---------- Paramètres ----------
CSV_PATH = "restaurant_count_by_locality.csv"   # <- chemin vers ton CSV
YEAR_MIN, YEAR_MAX = 2011, 2025                 # plage d'années à analyser (inclusives)
EXCLUDE_YEAR = []                               

# ---------- Chargement ----------
df = pd.read_csv(CSV_PATH)

# Normalisation de colonnes
df.columns = [c.strip().lower() for c in df.columns]
expected_cols = {"locality", "year", "num_restaurants"}
missing = expected_cols - set(df.columns)
if missing:
    raise ValueError(
        f"Colonnes manquantes dans le CSV: {missing}. "
        f"Colonnes trouvées: {list(df.columns)}"
    )

# Filtrage années et valeurs valides
mask = (df["year"].between(YEAR_MIN, YEAR_MAX)) & (~df["year"].isin(EXCLUDE_YEAR))
df = df.loc[mask].copy()
df = df[df["num_restaurants"] >= 0].copy()

# Exclure localités avec < 3 années distinctes
year_counts = df.groupby("locality")["year"].nunique()
excluded_low_years = year_counts[year_counts < 3].index.tolist()
df_fit = df[~df["locality"].isin(excluded_low_years)].copy()

failed_fits = []

def poisson_trend_for_group(g: pd.DataFrame) -> pd.Series:
    """
    Calcule la tendance via GLM Poisson (annual_growth_pct) et un score:
    trend_score = (Annual Growth %) * max(0, deviance_explained)
    Retourne uniquement locality et trend_score (les autres calculs restent internes).
    """
    g = (
        g.sort_values("year")
         .groupby(["locality", "year"], as_index=False)["num_restaurants"]
         .sum()
         .sort_values("year")
    )

    loc = g["locality"].iloc[0]
    first_year = int(g["year"].iloc[0])
    last_year  = int(g["year"].iloc[-1])
    n_years = last_year - first_year
    first_val = float(g["num_restaurants"].iloc[0])
    last_val  = float(g["num_restaurants"].iloc[-1])

    # Cas constant
    if g["num_restaurants"].nunique() == 1:
        return pd.Series({"locality": loc, "trend_score": 0.0})

    # Non-constant : fit GLM
    year_centered = g["year"] - g["year"].mean()
    X = pd.DataFrame({"const": 1.0, "year": year_centered.astype(float)})
    y = g["num_restaurants"].astype(float)

    annual_growth_pct = np.nan
    dev_explained = 0.0

    model = sm.GLM(y, X, family=sm.families.Poisson())
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            use_hc3 = (len(g) - X.shape[1]) > 0
            res = model.fit(cov_type="HC3") if use_hc3 else model.fit()

        beta_year = res.params.get("year", np.nan)
        if np.isfinite(beta_year):
            annual_growth_pct = (np.exp(beta_year) - 1.0) * 100.0

        dev = getattr(res, "deviance", np.nan)
        null_dev = getattr(res, "null_deviance", np.nan)
        if np.isfinite(dev) and np.isfinite(null_dev) and null_dev > 0:
            dev_explained = max(0.0, 1.0 - (dev / null_dev))

        trend_score = annual_growth_pct * dev_explained if np.isfinite(annual_growth_pct) else np.nan

    except (PerfectSeparationError, np.linalg.LinAlgError, ValueError) as e:
        failed_fits.append((loc, str(e)))
        trend_score = np.nan

    return pd.Series({"locality": loc, "trend_score": trend_score})

# ---------- Calcul par localité ----------
if df_fit.empty:
    trend_min = pd.DataFrame(columns=["locality", "trend_score"])
else:
    trend = (
        df_fit.groupby("locality", group_keys=False)
              .apply(poisson_trend_for_group)
              .reset_index(drop=True)
    )
    trend_min = trend.sort_values("trend_score", ascending=False)[["locality", "trend_score"]]

# ---------- Affichage et sauvegarde (seulement 2 colonnes) ----------
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")
print(trend_min.head(20).to_string(index=False))

OUT_CSV = "locality_trend_score.csv"
trend_min.to_csv(OUT_CSV, index=False)
print(f"\n📄 Export → {OUT_CSV}")


          locality  trend_score
         Rougemont     111.2918
Torricella-Taverne      74.8855
         Fontenais      37.7294
            Soazza      37.7294
           Bottens      37.1124
          Kappelen      36.6746
             Ayent      31.6811
           Conthey      26.9916
            Zernez      26.8036
   Collombey-Muraz      25.7476
         Deitingen      25.3202
Fischbach-Göslikon      24.8074
           Mesocco      24.1519
    Hauterive (NE)      23.6812
       Le Noirmont      23.1568
          Dierikon      23.1568
             Gland      23.1148
  Kleinandelfingen      22.8024
        Porrentruy      21.9411
         Oftringen      21.8580

📄 Export → locality_trend_score.csv


/tmp/ipykernel_53190/4181378250.py:106: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(poisson_trend_for_group)
